In [2]:
!git clone https://github.com/sathishkumar67/Face-Recognition-using-Resnet.git
!mv /kaggle/working/Face-Recognition-using-Resnet/* /kaggle/working/
!pip install --upgrade pip
!pip install -r requirements.txt
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Cloning into 'Face-Recognition-using-Resnet'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 59 (delta 24), reused 42 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 219.86 KiB | 9.99 MiB/s, done.
Resolving deltas: 100% (24/24), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.7 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 48.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.1 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 5.3 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.2 MB/s eta 0:00:00
 

In [4]:
import os
import random
import shutil
import cv2
from mtcnn import MTCNN
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset
from huggingface_hub import hf_hub_download
from siamese_resnet.utils import unzip_file
import torchvision.transforms as transforms
from tqdm import tqdm

In [12]:
DATASET_REPO_ID = "pt-sk/Face_Recognition_Dataset"
DATA_ROOT = "face_recognition_with_cropped_faces_dataset"
DATASET_FILENAME_IN_REPO = f"{DATA_ROOT}.zip"
DATASET_REPO_TYPE = "dataset"
LOCAL_DIR = os.getcwd()
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [8]:
# Download the dataset from Hugging Face Hub
hf_hub_download(repo_id=DATASET_REPO_ID, filename=DATASET_FILENAME_IN_REPO, repo_type=DATASET_REPO_TYPE, local_dir=LOCAL_DIR)

# Unzip the dataset
unzip_file(os.path.join(LOCAL_DIR, DATASET_FILENAME_IN_REPO), LOCAL_DIR)

(…)cognition_with_cropped_faces_dataset.zip:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 1.31G/1.31G [00:14<00:00, 90.1MB/s]


Unzipped /kaggle/working/face_recognition_with_cropped_faces_dataset.zip to /kaggle/working
Removed zip file: /kaggle/working/face_recognition_with_cropped_faces_dataset.zip


In [14]:
from siamese_resnet.dataset import TripletDataset, TripletDatasetGenerator

random.seed(SEED)
np.random.seed(SEED)

# Initialize generator
generator = TripletDatasetGenerator(DATA_ROOT)

# Create splits
splits = generator.create_splits()
    
# Generate triplets for each split
train_triplets = generator.generate_triplets(splits['train'])
val_triplets = generator.generate_triplets(splits['val'])
test_triplets = generator.generate_triplets(splits['test'])

# Create datasets
train_dataset = TripletDataset(train_triplets)
val_dataset = TripletDataset(val_triplets)
test_dataset = TripletDataset(test_triplets)
    
    # Create dataloaders (4 workers for optimal IO)
    # train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, 
    #                         num_workers=4, pin_memory=True)

In [ ]:

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, collate_fn=triplet_collate_fn)

In [75]:
a = next(iter(train_loader))

In [76]:
a["anchor"].shape, a["positive"].shape, a["negative"].shape

(torch.Size([1, 3, 224, 224]),
 torch.Size([1, 3, 224, 224]),
 torch.Size([1, 3, 224, 224]))

In [78]:
# import matplotlib.pyplot as plt
# plt.imshow(a["anchor"].permute(0, 2, 3, 1)[0].numpy())

In [58]:
a["anchor"]

tensor([[[[0.3725, 0.3608, 0.3373,  ..., 0.4784, 0.4941, 0.5020],
          [0.3529, 0.3490, 0.3451,  ..., 0.4392, 0.4667, 0.4863],
          [0.3569, 0.3569, 0.3529,  ..., 0.3961, 0.4235, 0.4392],
          ...,
          [0.6235, 0.5922, 0.5490,  ..., 0.2627, 0.2745, 0.2902],
          [0.6314, 0.5961, 0.5490,  ..., 0.2627, 0.2824, 0.3020],
          [0.6353, 0.6039, 0.5608,  ..., 0.2784, 0.3020, 0.3176]],

         [[0.2235, 0.2118, 0.1765,  ..., 0.3333, 0.3412, 0.3490],
          [0.2039, 0.2000, 0.1843,  ..., 0.2941, 0.3137, 0.3333],
          [0.2078, 0.2078, 0.2039,  ..., 0.2588, 0.2784, 0.2941],
          ...,
          [0.4431, 0.4118, 0.3686,  ..., 0.0941, 0.1137, 0.1412],
          [0.4431, 0.4078, 0.3647,  ..., 0.0941, 0.1333, 0.1608],
          [0.4471, 0.4157, 0.3765,  ..., 0.1098, 0.1529, 0.1765]],

         [[0.1412, 0.1294, 0.0980,  ..., 0.2627, 0.2745, 0.2824],
          [0.1216, 0.1176, 0.1059,  ..., 0.2235, 0.2471, 0.2667],
          [0.1333, 0.1333, 0.1294,  ..., 0

In [ ]:
# triplet loss function
# This function computes the triplet loss for a batch of anchor, positive, and negative samples.
class TripletLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        # Compute pairwise distances
        distance_positive = F.pairwise_distance(anchor, positive)
        distance_negative = F.pairwise_distance(anchor, negative)
        
        # Calculate triplet loss
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()
    


In [ ]:
class SiameseResNet(nn.Module):
    def __init__(self, embedding_dim=256):
        super(SiameseResNet, self).__init__()
        # Load pretrained ResNet18
        self.backbone = torchvision.models.resnet18(weights="IMAGENET1K_V1", progress=True)
        
        # Replace the final fully connected layer
        self.backbone.fc = nn.Linear(512, embedding_dim)  # 512 -> 256

    def forward(self, x):
        return self.backbone(x)
    
    def print_parameters_count(self):
        total_params = sum(p.numel() for p in self.parameters()) / 1e6  # Convert to millions
        # Print the number of parameters in millions
        print(f"Total parameters: {total_params:.2f}m")
    
model = SiameseResNet(embedding_dim=256)
# model.print_parameters_count()

# pass a sample image through the model
dummy_input = torch.randn(1, 3, 224, 224)  # batch size of 1, 3 channels, 224x224 image
output = model(dummy_input)
# output.shape  # should be (1, 256) since we changed the final layer to output 256 features

In [ ]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset
from collections import defaultdict

class TripletFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.id_to_images = defaultdict(list)
        
        # Populate identities and their images
        for identity in os.listdir(root_dir):
            identity_dir = os.path.join(root_dir, identity)
            if os.path.isdir(identity_dir):
                images = [os.path.join(identity_dir, img) 
                        for img in os.listdir(identity_dir) 
                        if img.endswith(('.jpg', '.png'))]
                if len(images) >= 2:  # Ensure at least 2 images per identity
                    self.id_to_images[identity] = images
        self.identities = list(self.id_to_images.keys())

    def __len__(self):
        return len(self.identities) * 10  # Adjust based on your needs

    def __getitem__(self, idx):
        # Anchor and positive from the same identity
        anchor_id = self.identities[idx % len(self.identities)]
        anchor_img_path, positive_img_path = random.sample(self.id_to_images[anchor_id], 2)
        
        # Negative from a different identity
        negative_id = random.choice(self.identities)
        while negative_id == anchor_id:
            negative_id = random.choice(self.identities)
        negative_img_path = random.choice(self.id_to_images[negative_id])
        
        # Load and transform images
        anchor = Image.open(anchor_img_path).convert('RGB')
        positive = Image.open(positive_img_path).convert('RGB')
        negative = Image.open(negative_img_path).convert('RGB')
        
        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)
        
        return anchor, positive, negative

In [6]:
train_dataset = TripletFaceDataset(root_dir='sample_dataset')

In [7]:
train_dataset.__len__()

3500

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

# Define transforms
transform = transforms.Compose([
    transforms.Resize(100),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize dataset and dataloader
dataset = TripletFaceDataset(root_dir="path/to/dataset", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = SiameseResNet(embedding_dim=256)
criterion = TripletLoss(margin=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):
    for batch in dataloader:
        anchor, positive, negative = batch
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        
        # Forward pass
        anchor_emb = model(anchor)
        positive_emb = model(positive)
        negative_emb = model(negative)
        
        # Compute loss
        loss = criterion(anchor_emb, positive_emb, negative_emb)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")